In [ ]:
with open("data/test.tsv", "r", encoding="utf-8") as f:
    dataset = f.read()
lines = dataset.split("\n")

for line in lines[1:-1]:
    # print(line.split('\t'))
    # print(line.split('\t')[0])
    # print(line.split('\t')[1].split(' '))
    label, text_a = line.split("\t")
    print(label,text_a)
    

In [7]:
import os
import numpy as np

import mindspore
from mindspore.dataset import text,GeneratorDataset
from mindspore import nn
from mindnlp.models import BertModel, BertConfig
from mindnlp.transforms.tokenizers import BertTokenizer
from mindnlp.transforms import PadTransform

class SentimentDataset:
    """Sentiment Dataset"""
    def __init__(self, path):
        self.path = path
        self._labels, self._text_a= [], []
        self._load()

    def _load(self):
        with open(self.path, "r", encoding="utf-8") as f:
            dataset = f.read()
        lines = dataset.split("\n")
        for line in lines[1:-1]:
            label, text_a = line.split("\t")
            self._labels.append(int(label))
            self._text_a.append(text_a)

    def __getitem__(self, index):
        return self._labels[index], self._text_a[index]

    def __len__(self):
        return len(self._labels)

column_names = ["label", "text_a"]

dataset_train = GeneratorDataset(source=SentimentDataset("data/train.tsv"), 
                                 column_names=column_names, shuffle=False)

dataset_val = GeneratorDataset(source=SentimentDataset("data/dev.tsv"),
                               column_names=column_names, shuffle=False)

dataset_test = GeneratorDataset(source=SentimentDataset("data/test.tsv"),
                                column_names=column_names, shuffle=False)

test = dataset_test.create_dict_iterator()

train = dataset_train.create_dict_iterator()

vocab_path = os.path.join("/home/tianyuzhou/.mindnlp/workflow/sentiment_analysis/bert", "vocab.txt")
vocab = text.Vocab.from_file(vocab_path)
pad_value_text = vocab.tokens_to_ids('[PAD]')

tokenizer = BertTokenizer(vocab)

# dataset_train = dataset_train.map(operations=[tokenizer], input_columns="text_a")

pad_op_text = PadTransform(max_length=128, pad_value=pad_value_text)

dataset_train = dataset_train.map(operations=[tokenizer,pad_op_text], input_columns="text_a")
dataset_train = dataset_train.batch(32)

train = dataset_train.create_dict_iterator()

# max_length = 0
# for i in train:
#     if len(i["text_a"]) > max_length:
#         max_length = len(i["text_a"])

# print(max_length)
next(train)

{'label': Tensor(shape=[32], dtype=Int64, value= [1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 
  1, 1, 1, 1, 1, 1, 1, 1]),
 'text_a': Tensor(shape=[32, 128], dtype=Int64, value=
 [[ 101,  872, 4638 ...    0,    0,    0],
  [ 101, 1420, 6432 ...    0,    0,    0],
  [ 101,  872, 2823 ...    0,    0,    0],
  ...
  [ 101, 2769, 3291 ...    0,    0,    0],
  [ 101, 1520, 1520 ...    0,    0,    0],
  [ 101, 4130, 2404 ...    0,    0,    0]])}